In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("../library")
import RecSys as rs
#patch = "" # For Google CoLab
path = "../data/"

import collections

In [2]:
chefmozacccepts = pd.read_csv(path+"chefmozaccepts.csv")
chefmozcuisine = pd.read_csv(path+"chefmozcuisine.csv")
chefmozhours4 = pd.read_csv(path+"chefmozhours4.csv")
chefmozparking = pd.read_csv(path+"chefmozparking.csv")
geoplaces2 = pd.read_csv(path+"geoplaces2.csv")
rating_final = pd.read_csv(path+"rating_final.csv")
usercuisine = pd.read_csv(path+"usercuisine.csv")
userpayment = pd.read_csv(path+"userpayment.csv")
userprofile = pd.read_csv(path+"userprofile.csv")
# This will contain the list of Datas that we will 
# use for the Recommendation System
listOfData_r = [rating_final]

# This doesn't make much sense, we can split the Clasification for have less dummy variables, 
# but we will try make the recomendation with full data
chefmozacccepts['Rpayment_accepts'] = chefmozacccepts['Rpayment']
chefmozacccepts['Rpayment_accepts'] = chefmozacccepts['Rpayment_accepts']\
             .replace(['VISA', 'MasterCard-Eurocard', 'American_Express',
                       'bank_debit_cards','Discover', 'Carte_Blanche',
                       'Diners_Club', 'Visa', 'Japan_Credit_Bureau'], ' credictCard')
chefmozacccepts['Rpayment_accepts'] = chefmozacccepts['Rpayment_accepts']\
             .replace(['gift_certificates'], 'checks')
chefmozacccepts['Rpayment_accepts'] = chefmozacccepts['Rpayment_accepts'].astype('category')

chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine']
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['French','Spanish','Italian','Seafood','Greek','Mediterranean','Portuguese','Lebanese', 'Moroccan','Israeli','Tapas','Basque','Tunisian','Turkish','North_African'], 'Mediterranean')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Latin_American','Mexican','Brazilian','Caribbean','Chilean','Filipino','Cuban','Hawaiian','Peruvian','Jamaican','Southern','Regional','Indigenous'], 'South_American')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Fast_Food','Hot_Dogs','Burgers','Deli-Sandwiches','Pizzeria'], 'Fast_Food')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Asian','Mongolian','Chinese','Japanese','Sushi','Vietnamese', 'Korean','Thai','Indian-Pakistani','Tibetan','Southeast_Asian','Burmese','Dim_Sum','Malaysian','Cambodian'], 'Asian')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['German','Polish', 'Dutch-Belgian','Continental-European','Eastern_European','Irish','Hungarian','British','Romanian','Austrian','Scandinavian','Swiss','Russian-Ukrainian'], 'European')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Afghan','Armenian','Persian','Middle_Eastern'], 'Middle East')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['African','Ethiopian'], 'African')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Breakfast-Brunch','Bagels','Juice'], 'Breakfast')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Bar_Pub_Brewery','Bar'], 'Bar')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Cafeteria','Cafe-Coffee_Shop','Tea_House'], 'Coffee_Shop')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Fine_Dining','Organic-Healthy','Vegetarian'], 'Healthy-Vegetarian')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['American','California','Southwestern','Canadian','Cajun-Creole','Pacific_Northwest','Tex-Mex'], 'North_American')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Steaks','Barbecue'], 'Meat')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Dessert-Ice_Cream'], 'Ice_Cream')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Doughnuts','Bakery'], 'Bakery')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Contemporary','Fusion','Eclectic'], 'Contemporary')
chefmozcuisine['Rcuisine_Type'] = chefmozcuisine['Rcuisine_Type'].replace(['Australian','Polynesian','Indonesian','Pacific_Rim'], 'Australian')

chefmozhours4['days'] = chefmozhours4['days'].replace(['Mon;Tue;Wed;Thu;Fri;'],'Diary_Days')

chefmozparking['parking'] = chefmozparking['parking_lot']
chefmozparking['parking'] = chefmozparking['parking'].replace(['yes','public', 'valet parking','validated parking','fee'], 'yes')
chefmozparking['parking'] = chefmozparking['parking'].replace(['none','street'], 'no')

geoplaces2['have_smoking_area']=geoplaces2['smoking_area']
geoplaces2['have_smoking_area']=geoplaces2['have_smoking_area'].replace(['only at bar', 'permitted', 'section'],'yes')
geoplaces2['have_smoking_area']=geoplaces2['have_smoking_area'].replace(['none','not permitted'],'no')

geoplaces2['accessibility'] = geoplaces2['accessibility']
geoplaces2['accessibility'] = geoplaces2['accessibility'].replace(['completely', 'partially'],'yes_accessibility')

geoplaces2['franchise']=geoplaces2['franchise'].replace('t','no')
geoplaces2['franchise']=geoplaces2['franchise'].replace('f','yes')

userpayment['Upayment_accepts'] = userpayment['Upayment']
userpayment['Upayment_accepts'] = userpayment['Upayment_accepts']\
             .replace(['VISA', 'MasterCard-Eurocard', 'American_Express',
                       'bank_debit_cards','Discover', 'Carte_Blanche',
                       'Diners_Club', 'Visa', 'Japan_Credit_Bureau'], 'credictCard')
userpayment['Upayment_accepts'] = userpayment['Upayment_accepts']\
             .replace(['gift_certificates'], 'checks')
userpayment['Upayment_accepts'] = userpayment['Upayment_accepts'].astype('category')

usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine']
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['French','Spanish','Italian','Seafood','Greek','Mediterranean','Portuguese','Lebanese', 'Moroccan','Israeli','Tapas','Basque','Tunisian','Turkish','North_African'], 'Mediterranean')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Latin_American','Mexican','Brazilian','Caribbean','Chilean','Filipino','Cuban','Hawaiian','Peruvian','Jamaican','Southern','Regional','Indigenous'], 'South_American')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Fast_Food','Hot_Dogs','Burgers','Deli-Sandwiches','Pizzeria'], 'Fast_Food')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Asian','Mongolian','Chinese','Japanese','Sushi','Vietnamese', 'Korean','Thai','Indian-Pakistani','Tibetan','Southeast_Asian','Burmese','Dim_Sum','Malaysian','Cambodian'], 'Asian')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['German','Polish', 'Dutch-Belgian','Continental-European','Eastern_European','Irish','Hungarian','British','Romanian','Austrian','Scandinavian','Swiss','Russian-Ukrainian'], 'European')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Afghan','Armenian','Persian','Middle_Eastern'], 'Middle East')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['African','Ethiopian'], 'African')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Breakfast-Brunch','Bagels','Juice'], 'Breakfast')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Bar_Pub_Brewery','Bar'], 'Bar')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Cafeteria','Cafe-Coffee_Shop','Tea_House'], 'Coffee_Shop')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Fine_Dining','Organic-Healthy','Vegetarian'], 'Healthy-Vegetarian')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['American','California','Southwestern','Canadian','Cajun-Creole','Pacific_Northwest','Tex-Mex'], 'North_American')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Steaks','Barbecue'], 'Meat')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Dessert-Ice_Cream'], 'Ice_Cream')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Doughnuts','Bakery'], 'Bakery')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Contemporary','Fusion','Eclectic'], 'Contemporary')
usercuisine['Rcuisine_Type'] = usercuisine['Rcuisine_Type'].replace(['Australian','Polynesian','Indonesian','Pacific_Rim'], 'Australian')





rs.replaceID(realID=132706,falseID=132584,ID='placeID',data=listOfData_r)

In [3]:
listOfData = [chefmozacccepts,chefmozcuisine,chefmozhours4,
              chefmozparking,geoplaces2,rating_final,
              usercuisine,userpayment,userprofile]

In [4]:
### We have to do the secondview clean

In [5]:
rs.replaceID(realID=132706,falseID=132584,ID='placeID',data=listOfData_r)

In [6]:
###############
# Those will be the data that we will use for the model and app
###############
userProfile_r = userprofile[['userID', 'smoker', 'drink_level','dress_preference', 
                             'ambience', 'transport', 'marital_status','hijos', 
                             'religion', 'activity']]

usercuisine_list = usercuisine.groupby('userID').apply(lambda x: list(x['Rcuisine_Type']))
usercuisine_list = pd.DataFrame({'userID':usercuisine_list.index,'Rcuisine_Type':usercuisine_list.values})
userProfile_r = userProfile_r.merge(usercuisine_list, how = 'left', on = 'userID')

userpayment_list= userpayment.groupby('userID').apply(lambda x: list(x['Upayment_accepts']))
userpayment_list = pd.DataFrame({'userID':userpayment_list.index,'Upayment_accepts':userpayment_list.values})
userProfile_r = userProfile_r.merge(userpayment_list, how = 'left', on = 'userID')

userGeo_r = userprofile[['userID', 'latitude', 'longitude']]

userChefmozRelation_r = rating_final
userChefmozRelation_r = userChefmozRelation_r.merge(userpayment, how='left', on='userID')
userChefmozRelation_r = userChefmozRelation_r.merge(usercuisine, how='left', on='userID')

chefmozProfile_r = geoplaces2[['placeID', 'alcohol', 'smoking_area','dress_code',
                               'accessibility', 'price', 'Rambience', 'franchise',
                               'area', 'other_services']]

chefmozProfile_r= chefmozProfile_r.merge(chefmozacccepts, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozcuisine, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozhours4, how = 'left', on = 'placeID')
chefmozProfile_r= chefmozProfile_r.merge(chefmozparking, how = 'left', on = 'placeID')

chefmozGeo_r = geoplaces2[['placeID', 'latitude', 'longitude', 'the_geom_meter',
                           'address', 'city', 'state', 'country', 'fax', 'zip']]

listPlaceID2Drop = list(chefmozProfile_r[chefmozProfile_r["Rcuisine_Type"]=='nan']['placeID'].values)
rating_final = rating_final.drop(rating_final['placeID'].isin(listPlaceID2Drop).index)
chefmozGeo_r = chefmozGeo_r.drop(chefmozGeo_r['placeID'].isin(listPlaceID2Drop).index)
userChefmozRelation_r = userChefmozRelation_r.drop(userChefmozRelation_r['placeID'].isin(listPlaceID2Drop).index)
chefmozProfile_r = chefmozProfile_r.drop(chefmozProfile_r['placeID'].isin(listPlaceID2Drop).index)

In [7]:
l,df = rs.checkData(listOfData,'?')

In [8]:
l

[(4, 'address'),
 (4, 'city'),
 (4, 'state'),
 (4, 'country'),
 (4, 'fax'),
 (4, 'zip'),
 (4, 'url'),
 (8, 'smoker'),
 (8, 'dress_preference'),
 (8, 'ambience'),
 (8, 'transport'),
 (8, 'marital_status'),
 (8, 'hijos'),
 (8, 'activity'),
 (8, 'budget')]

In [9]:
len(userprofile['userID'].unique())==len(userprofile['userID'])

True

In [10]:
len(geoplaces2['placeID'].unique())==len(geoplaces2['placeID'])

True

In [11]:
len(chefmozparking['placeID'].unique())==len(chefmozparking['placeID'])

False

In [12]:
duplicate = [item for item, count in collections.Counter(chefmozparking['placeID']).items() if count > 1]
chefmozparking[chefmozparking['placeID'].isin(duplicate)].head(5)

,placeID,parking_lot,parking
615,132557,public,yes
616,132557,street,no
619,132531,public,yes
620,132531,street,no
621,132530,fee,yes


In [13]:
len(chefmozhours4['placeID'].unique())==len(chefmozhours4['placeID'])

False

In [14]:
duplicate = [item for item, count in collections.Counter(chefmozhours4['placeID']).items() if count > 1]
chefmozhours4[chefmozhours4['placeID'].isin(duplicate)].head(5)

,placeID,hours,days
0,135111,00:00-23:30;,Diary_Days
1,135111,00:00-23:30;,Sat;
2,135111,00:00-23:30;,Sun;
3,135110,08:00-19:00;,Diary_Days
4,135110,00:00-00:00;,Sat;


In [15]:
len(chefmozcuisine['placeID'].unique())==len(chefmozcuisine['placeID'])

False

In [16]:
duplicate = [item for item, count in collections.Counter(chefmozcuisine['placeID']).items() if count > 1]
print(duplicate)
chefmozcuisine[chefmozcuisine['placeID'].isin(duplicate)].head(5)

[135103, 135102, 135101, 135100, 135099, 135098, 135097, 135092, 135091, 135086, 135073, 135071, 135053, 135052, 135043, 135041, 135032, 135030, 135029, 135026, 135020, 135007, 135004, 134997, 134990, 134983, 134982, 134976, 134972, 132962, 132917, 132867, 132866, 132852, 132851, 132774, 132773, 132555, 132531, 132530, 132527, 132521, 132493, 132491, 132483, 132474, 132449, 132443, 132442, 132439, 132419, 132417, 132413, 132392, 132341, 132333, 132329, 132328, 132319, 132318, 132317, 132312, 132302, 132296, 132289, 132284, 132273, 132258, 132255, 132253, 132246, 132244, 132243, 132240, 132238, 132237, 132235, 132228, 132226, 132223, 132217, 132216, 132214, 132207, 132187, 132186, 132184, 132182, 132177, 132171, 132166, 132161, 132155, 132147, 132137, 132128, 132126, 132125, 132109, 132108, 132101, 132100, 132031, 132030, 132028, 132024, 132023, 132012, 132010, 132005]


,placeID,Rcuisine,Rcuisine_Type
6,135103,Burgers,Fast_Food
7,135103,Dessert-Ice_Cream,Ice_Cream
8,135103,Fast_Food,Fast_Food
9,135103,Hot_Dogs,Fast_Food
10,135102,Latin_American,South_American


In [17]:
len(chefmozacccepts['placeID'].unique())==len(chefmozacccepts['placeID'])

False

In [18]:
duplicate = [item for item, count in collections.Counter(chefmozacccepts['placeID']).items() if count > 1]
print(duplicate)
chefmozacccepts[chefmozacccepts['placeID'].isin(duplicate)].head(5)

[135110, 135107, 135106, 135104, 135103, 135102, 135099, 135098, 135097, 135093, 135091, 135089, 135086, 135084, 135081, 135080, 135079, 135078, 135076, 135074, 135073, 135072, 135071, 135068, 135067, 135066, 135065, 135064, 135061, 135058, 135057, 135055, 135054, 135053, 135052, 135050, 135048, 135047, 135046, 135045, 135040, 135037, 135035, 135034, 135033, 135032, 135031, 135029, 135028, 135027, 135025, 135024, 135018, 135008, 135007, 135006, 135004, 135002, 135001, 134994, 134993, 134992, 134991, 134989, 134988, 134985, 134984, 134983, 134978, 134977, 134976, 134975, 134974, 134973, 134972, 134971, 134970, 133043, 133033, 133030, 133027, 133026, 132958, 132957, 132956, 132955, 132954, 132953, 132947, 132931, 132927, 132917, 132900, 132892, 132888, 132887, 132886, 132880, 132875, 132873, 132872, 132863, 132862, 132861, 132860, 132859, 132857, 132854, 132835, 132826, 132824, 132820, 132811, 132808, 132807, 132806, 132804, 132802, 132801, 132799, 132797, 132796, 132795, 132793, 132792,

,placeID,Rpayment,Rpayment_accepts
0,135110,cash,cash
1,135110,VISA,credictCard
2,135110,MasterCard-Eurocard,credictCard
3,135110,American_Express,credictCard
4,135110,bank_debit_cards,credictCard


In [19]:
chefmozProfile_r[chefmozProfile_r['placeID']==132584]

,placeID,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services,Rpayment,Rpayment_accepts,Rcuisine,Rcuisine_Type,hours,days,parking_lot,parking


In [20]:
chefmozProfile_r[chefmozProfile_r['placeID']==132706]

,placeID,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services,Rpayment,Rpayment_accepts,Rcuisine,Rcuisine_Type,hours,days,parking_lot,parking


In [21]:
chefmozGeo_r[chefmozGeo_r['placeID']==132706]

,placeID,latitude,longitude,the_geom_meter,address,city,state,country,fax,zip


In [22]:
chefmozGeo_r[chefmozGeo_r['placeID']==132584]

,placeID,latitude,longitude,the_geom_meter,address,city,state,country,fax,zip


In [23]:
chefmozProfile_r[chefmozProfile_r['placeID']==132706]

,placeID,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services,Rpayment,Rpayment_accepts,Rcuisine,Rcuisine_Type,hours,days,parking_lot,parking


In [24]:
chefmozProfile_r = rs.unionData(chefmozProfile_r,'placeID')

Exception: Data must be 1-dimensional

In [ ]:
chefmozProfile_r.columns

In [ ]:
columns = ['alcohol', 'smoking_area', 'dress_code', 'accessibility',
       'price', 'Rambience', 'franchise', 'area', 'other_services', 'Rpayment',
       'Rpayment_accepts', 'Rcuisine', 'Rcuisine_Type', 'hours', 'days',
       'parking_lot', 'parking']
for col in columns:
    for i in chefmozProfile_r[col]:
            print(col)

In [ ]:
chefmozProfile_r['alcohol']

In [ ]:
columns = ['alcohol', 'smoking_area', 'dress_code', 'accessibility',
       'price', 'Rambience', 'franchise', 'area', 'other_services', 'Rpayment',
       'Rpayment_accepts', 'Rcuisine', 'Rcuisine_Type', 'hours', 'days',
       'parking_lot', 'parking']
for col in columns:
    print(col)
    print(chefmozProfile_r[col].unique())

In [ ]:
userProfile_r.head()

In [ ]:
chefmozProfile_r.head()

In [ ]:
listPlaceID2Drop = list(chefmozProfile_r[chefmozProfile_r["Rcuisine_Type"]=='nan']['placeID'].values)

In [ ]:
chefmozcuisine[chefmozcuisine['placeID']==135040]

In [ ]:
geoplaces2[geoplaces2['placeID'].isin(l)]

In [ ]:
rating_final[rating_final['placeID']==135040]

In [ ]:
rating_final = rating_final.drop(rating_final['placeID'].isin(listPlaceID2Drop).index)
geoplaces2 = geoplaces2.drop(geoplaces2['placeID'].isin(listPlaceID2Drop).index)
rating_final = rating_final.drop(rating_final['placeID'].isin(listPlaceID2Drop).index)
rating_final = rating_final.drop(rating_final['placeID'].isin(listPlaceID2Drop).index)
rating_final = rating_final.drop(rating_final['placeID'].isin(listPlaceID2Drop).index)

In [ ]:
rating_final[rating_final['placeID'].isin(listPlaceID2Drop)]